In [7]:
"""
Steps to replicate derived property in holding domain on aggregation api returns different value. 

1. Define portfolio property. (which the derived property depends on)
2. Create portfolio with property.
3. upsert instruments.
4. Upsert transaction.
5. Upsert quote.
6. Upsert recipe.
7. Create holding derv property.
8. Do valuation to check values from these properties.
9. Check against holding API.

"""


"""
------------------------
Creating an API Factory:
------------------------

"""

# Import LUSID SDK
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory

# Import supporting LUSID utility packages
from lusidjam import RefreshingToken
#from fbnsdkutilities import ApiClientFactory

# Import other non-LUSID packages
import json
import pandas as pd
from datetime import datetime, timedelta, timezone
import os


# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")

print(f'Secrets path is {secrets_path}')

# Run the Notebook locally using a secrets file (see https://support.lusid.com/knowledgebase/article/KA-01663)
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

api_factory = ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="VSCode"
)

print('LUSID Environment Initialised')

# Define a portfolio and parameters.
portfolio_code = "DERV_PROP_TEST_PORTFOLIO"
portfolio_name = "Derived Property Test Portfolio"

scope = "DERV_PROP_TEST"
transaction_scope = "DERV_PROP_TEST"
domain = "Portfolio"

# Time and date parameters in normalised format.
date_format = '%Y-%m-%d'


"""
---------------
Create Property
---------------

"""

# Create portfolio property.

properties_api = api_factory.build(lusid.api.PropertyDefinitionsApi)
portfolio_scope = "Test"

properties = ["Schema"]

for property_code in properties:
    try:
        property_creation = properties_api.create_property_definition(
            create_property_definition_request=models.CreatePropertyDefinitionRequest(
                domain="Portfolio",
                scope=portfolio_scope,
                code=property_code,
                display_name=property_code,
                data_type_id=models.ResourceId(code="string", scope="system"),
            )
        )

        print(property_creation)

    except lusid.ApiException as e:
        print(json.loads(e.body)["title"])


"""

-------------------------------
Create a Transaction Portfolio:
-------------------------------

"""

portfolio_creation_date = datetime.today() + timedelta(days=-3)

# Create a Transactions Portfolios API
transaction_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)

# Attempt to create a Transaction Portfolio
try:
    create_portfolio_response = transaction_portfolio_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=models.CreateTransactionPortfolioRequest(
            display_name=portfolio_name,
            code=portfolio_code,
            description=portfolio_name,
            base_currency="GBP",
            created=portfolio_creation_date.strftime(date_format),
            properties={
                'Portfolio/Test/Schema': models.ModelProperty(
                    key = 'Portfolio/Test/Schema',
                    value = models.PropertyValue(
                        label_value = 'TestSchema'
                    )
                )
            },
        ),
    )

    print(create_portfolio_response)

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

Secrets path is C:/Users/JoeCockings/OneDrive - Finbourne Technology Limited/Repos/secrets.json
LUSID Environment Initialised
Error creating Property Definition 'Portfolio/Test/Schema' because it already exists.
Could not create a portfolio with id 'DERV_PROP_TEST_PORTFOLIO' because it already exists in scope 'DERV_PROP_TEST'.


In [91]:
"""
------------------------
Create Equity Instrument
------------------------

"""

test_equity_id = "DPTESTINST01"

instruments_api = api_factory.build(lusid.api.InstrumentsApi)

equity_definition = models.Equity(
    identifiers={},
    dom_ccy="GBP",
    instrument_type="Equity"
)

upsert_instrument = instruments_api.upsert_instruments(
    request_body={
        "test_1": models.InstrumentDefinition(
            name="TEST EQUITY",
            identifiers={
                "ClientInternal": models.InstrumentIdValue(
                    value=test_equity_id
                )
            },
            definition=equity_definition,
        )
    }
)

print(upsert_instrument)


{'failed': {},
 'href': None,
 'links': [{'description': None,
            'href': 'https://fbn-joe.lusid.com/api/api/schemas/entities/UpsertInstrumentsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'https://fbn-joe.lusid.com/app/insights/logs/0HN463G04KSQC:00000087',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'metadata': {'actions': [{'description': 'The request identifiers of Unchanged Instruments',
 'identifier_type': 'RequestId',
 'identifiers': ['test_1'],
 'type': 'UnchangedInstruments'}]},
 'values': {'test_1': {'asset_class': 'Equities',
                       'dom_ccy': 'GBP',
                       'href': 'https://fbn-joe.lusid.com/api/api/instruments/LusidInstrumentId/LUID_00003D8U?scope=default',
                       'identifiers': {'ClientInternal': 'DPT

In [92]:
"""
------------------
Upsert Transaction
------------------

"""

# Create a Transaction Portfolios API
transaction_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)

txn_date = datetime.today() + timedelta(days=-3)
settle_date = txn_date + timedelta(days=2)

try:
    upsert_transaction_response = transaction_portfolio_api.upsert_transactions(
        scope=scope,
        code=portfolio_code,
        transaction_request=[
            models.TransactionRequest(
                transaction_id="txn_001",
                type="StockIn",
                instrument_identifiers={"Instrument/default/ClientInternal": test_equity_id},
                transaction_date=txn_date.strftime(date_format),
                settlement_date=settle_date.strftime(date_format),
                units=100001,
                transaction_price=models.TransactionPrice(
                    price=101,
                    type="Price"
                ),
                total_consideration=models.CurrencyAndAmount(
                    amount=100000, currency="GBP"
                ),
            )
        ],
    )

    print(upsert_transaction_response)

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

{'href': 'https://fbn-joe.lusid.com/api/api/transactionportfolios/DERV_PROP_TEST/DERV_PROP_TEST_PORTFOLIO/transactions?asAt=2024-06-06T11%3A58%3A41.3371330%2B00%3A00',
 'links': [{'description': None,
            'href': 'https://fbn-joe.lusid.com/api/api/portfolios/DERV_PROP_TEST/DERV_PROP_TEST_PORTFOLIO?effectiveAt=2024-06-03T00%3A00%3A00.0000000%2B00%3A00&asAt=2024-06-06T11%3A58%3A41.3371330%2B00%3A00',
            'method': 'GET',
            'relation': 'Root'},
           {'description': None,
            'href': 'https://fbn-joe.lusid.com/api/api/schemas/entities/UpsertPortfolioTransactionsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'https://fbn-joe.lusid.com/app/insights/logs/0HN463GBIU8HS:00000056',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'metadata': {},


In [93]:
"""
-------------
Upsert Quotes
-------------

"""

quote_date = datetime.today() + timedelta(days=-1)

quotes_api = api_factory.build(lusid.api.QuotesApi)

# Create quote requests
instrument_quotes = {
    "upsert_request_1": models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            quote_series_id=models.QuoteSeriesId(
                provider="Client",
                instrument_id=test_equity_id,
                instrument_id_type="ClientInternal",
                quote_type="Price",
                field="mid",
            ),
            effective_at=quote_date.strftime(date_format),
        ),
        metric_value=models.MetricValue(value=12, unit="GBP"),
    )
}

try:
    # Upsert the quotes into LUSID
    response = quotes_api.upsert_quotes(
        scope=scope,
        request_body=instrument_quotes
    )

    print(response)

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

{'failed': {},
 'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'https://fbn-joe.lusid.com/app/insights/logs/0HN463G04KSSS:0000000D',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'values': {'upsert_request_1': {'as_at': datetime.datetime(2024, 6, 6, 11, 58, 43, 189512, tzinfo=tzlocal()),
                                 'cut_label': '',
                                 'lineage': '',
                                 'metric_value': {'unit': 'GBP', 'value': 12.0},
                                 'quote_id': {'effective_at': '2024-06-05T00:00:00.0000000+00:00',
                                              'quote_series_id': {'field': 'mid',
                                                                  'instrument_id': 'DPTESTINST01',
                                                                  'instrument_id_type': 'ClientInterna

In [94]:
"""
-------------
Upsert Recipe
-------------

"""

# Create a recipe to perform a valuation
recipe_code = "marketValue"

configuration_recipe = models.ConfigurationRecipe(
    scope=scope,
    code=recipe_code,
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
                key="Quote.ClientInternal.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
                quote_interval="10D.0D",
            ),
            models.MarketDataKeyRule(
                key="FX.*.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Rate",
                field="mid",
                quote_interval="10D.0D",
            ),
        ],
        suppliers=models.MarketContextSuppliers(
            commodity="Client",
            credit="Client",
            equity="Client",
            fx="Client",
            rates="Client",
        ),
        options=models.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="ClientInternal",
            default_scope=scope,
            attempt_to_infer_missing_fx=True,
        ),
    ),
    pricing=models.PricingContext(
        
        model_rules=[models.VendorModelRule(supplier="Lusid",
            model_name="SimpleStatic",
            instrument_type="Bond")]
       
        )
    )

configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)

upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )
)

print(upsert_configuration_recipe_response)

{'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'https://fbn-joe.lusid.com/app/insights/logs/0HN463G1H60QC:00000024',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'value': datetime.datetime(2024, 6, 6, 11, 58, 46, 997965, tzinfo=tzlocal())}


In [95]:
# Derived property.

derv_scope = "Test2"

create_derived_property_definition_request = {
    "domain":"Holding",
    "scope":derv_scope,
    "code":"AccountSchema",
    "displayName":"Account Schema",
    "dataTypeId":{
        "scope":"system",
        "code":"string"
    },
    "propertyDescription":"Schema of the underlying account",
    "derivationFormula":"coalesce(Properties[Portfolio/Test/Schema], '?')"
}

try:
    request = properties_api.create_derived_property_definition(
        create_derived_property_definition_request
    )

    print(request)

except:
    print(request)

{'code': 'AccountSchema',
 'collection_type': None,
 'constraint_style': None,
 'data_type_id': {'code': 'string', 'scope': 'system'},
 'derivation_formula': "coalesce(Properties[Portfolio/Test/Schema], '?')",
 'display_name': 'Account Schema',
 'domain': 'Holding',
 'href': None,
 'key': 'Holding/Test2/AccountSchema',
 'life_time': 'Perpetual',
 'links': [{'description': None,
            'href': 'https://fbn-joe.lusid.com/api/api/schemas/entities/PropertyDefinition',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'https://fbn-joe.lusid.com/app/insights/logs/0HN461NLDSNK7:0000011A',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'properties': {},
 'property_definition_type': 'DerivedDefinition',
 'property_description': 'Schema of the underlying account',
 'scope': 'Test2',
 'type': 

In [96]:
# Get valuation, see the properties are different values.

aggregation_api = api_factory.build(lusid.api.AggregationApi)

valuation_from = datetime.today() + timedelta(days=-1)
valuation_to = datetime.today()

valuation_request = {
    "recipeId": {
        "scope": scope,
        "code": recipe_code
    },
    "metrics": [
        {
            "key": "Portfolio/Test/Schema",
            "op": "Value",
            "options": {}
        },
        {
            "key": "Holding/Test2/AccountSchema",
            "op": "Value",
            "options": {}
        },
        {
            "key": "Valuation/PV",
            "op": "Value",
            "options": {}
        }
    ],
    "groupBy": [
        "Instrument/default/Name"
    ],
    "portfolioEntityIds": [
        {
            "scope": scope,
            "code": portfolio_code,
            "portfolioEntityType": "SinglePortfolio"
        }
    ],
    "valuationSchedule": {
        "effectiveFrom": valuation_from.strftime(date_format),
        "effectiveAt": valuation_to.strftime(date_format),
        "tenor": "1D",
        "rollConvention": "None",
        "holidayCalendars": [],
        "valuationDateTimes": [],
        "businessDayConvention": "F"
    }
}

try:
    # GetValuation: Perform valuation for a list of portfolios and/or portfolio groups
    api_response = aggregation_api.get_valuation(valuation_request=valuation_request)
    pprint.pprint(api_response.data)
    
except lusid.ApiException as e:
    pprint.pprint("Exception when calling AggregationApi->get_valuation: %s\n" % e)

[{'Holding/Test2/AccountSchema': '?',
  'Portfolio/Test/Schema': 'TestSchema',
  'Valuation/PV': 1200012.0}]


In [100]:
# As a control, check derived value via holdings API. On holdings the properties are the same.

try:
    respose = transaction_portfolio_api.get_holdings(
        scope, 
        portfolio_code, 
        effective_at=datetime.now(tz=timezone.utc).isoformat(), 
        as_at=datetime.now(tz=timezone.utc).isoformat(), 
        property_keys=[
            "Portfolio/Test/Schema",
            "Holding/Test2/AccountSchema"
        ],
    )

    pprint.pprint(respose.values[0].properties)

except lusid.ApiException as e:
    pprint.pprint("Exception when calling AggregationApi->get_valuation: %s\n" % e)

{'Holding/Test2/AccountSchema': {'effective_from': datetime.datetime(1, 1, 1, 0, 0, tzinfo=tzlocal()),
 'effective_until': None,
 'key': 'Holding/Test2/AccountSchema',
 'value': {'label_value': 'TestSchema',
           'label_value_set': None,
           'metric_value': None}},
 'Holding/default/SourcePortfolioId': {'effective_from': datetime.datetime(1, 1, 1, 0, 0, tzinfo=tzlocal()),
 'effective_until': None,
 'key': 'Holding/default/SourcePortfolioId',
 'value': {'label_value': 'DERV_PROP_TEST_PORTFOLIO',
           'label_value_set': None,
           'metric_value': None}},
 'Holding/default/SourcePortfolioScope': {'effective_from': datetime.datetime(1, 1, 1, 0, 0, tzinfo=tzlocal()),
 'effective_until': None,
 'key': 'Holding/default/SourcePortfolioScope',
 'value': {'label_value': 'DERV_PROP_TEST',
           'label_value_set': None,
           'metric_value': None}},
 'Portfolio/Test/Schema': {'effective_from': datetime.datetime(1, 1, 1, 0, 0, tzinfo=tzlocal()),
 'effective_until'